In [1]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import pandas as pd
import eipy.ei as e
from eipy.additional_ensembles import MeanAggregation, CES

In [2]:
from eipy.metrics import fmax_score
from sklearn.metrics import roc_auc_score, matthews_corrcoef

metrics = {
            'f_max': fmax_score,
            'auc': roc_auc_score,
            'mcc': matthews_corrcoef
            }

In [3]:
base_predictors = {
                    'ADAB': AdaBoostClassifier(),
                    'XGB': XGBClassifier(),
                    'DT': DecisionTreeClassifier(),
                    'RF': RandomForestClassifier(),
                    'GB': GradientBoostingClassifier(),
                    'KNN': KNeighborsClassifier(),
                    'LR': LogisticRegression(),
                    'NB': GaussianNB(),
                    'MLP': MLPClassifier(),
                    'SVM': SVC(probability=True),
}

In [4]:
import pickle as pkl
with open("/home/opc/eipy/tadpole/tadpole_data_time_imptn_norm_v1.pickle", "rb") as file:
    data = pkl.load(file)
with open("/home/opc/eipy/tadpole/tadpole_labels_time_imptn_norm_v1.pickle", "rb") as file:
    labels = pkl.load(file)

In [5]:
data = [data[k] for k in data.keys()]

In [6]:
for k,v in labels.items():
    labels[k] = v.reset_index(drop=True)

In [7]:
#intermediate transformation to make sure labels are ordered correctly in time
labels = pd.DataFrame(labels)

labels = labels.to_numpy()

In [8]:
import numpy as np
# encoding_dict = {'NL': 0, 'MCI': 1, 'Dementia': 2}
encoding_dict = {'NL': 0, 'MCI': 1, 'Dementia': 1} # for testsing binary

# Use numpy.vectorize with a lambda function to apply the encoding
labels = np.vectorize(lambda x: encoding_dict[x])(labels)

In [9]:
labels.shape

(702, 5)

In [10]:
for i in range(5):
    print(pd.Series(labels[:,i]).value_counts())

1    502
0    200
Name: count, dtype: int64
1    507
0    195
Name: count, dtype: int64
1    510
0    192
Name: count, dtype: int64
1    514
0    188
Name: count, dtype: int64
1    526
0    176
Name: count, dtype: int64


In [11]:
'''misalign data'''
data = data[1:]
labels = labels[:,:-1]

In [12]:
len(data), labels.shape

(4, (702, 4))

In [13]:
meta_data = []
for t in range(len(data)):
    #time dependent data splitting
    X_train_test_timestep = data[t]
    labels_at_timestep = labels[:, t]
    EI_for_timestep = e.EnsembleIntegration(
                        base_predictors=base_predictors,
                        k_outer=5,
                        k_inner=5,
                        n_samples=1,
                        sampling_strategy=None,
                        sampling_aggregation="mean",
                        n_jobs=-1,
                        metrics=metrics,
                        random_state=38,
                        project_name=f"time step {t}",
                        model_building=False,
                        )
    print(f"generating metadata for timestep {t}")
    EI_for_timestep.fit_base(X_train_test_timestep, labels_at_timestep)
    meta_data.append([EI_for_timestep.ensemble_training_data, EI_for_timestep.ensemble_test_data, EI_for_timestep.ensemble_training_data_final, EI_for_timestep.base_summary])


generating metadata for timestep 0
Training base predictors on Main cognitive tests...
        
... for ensemble performance analysis...


Generating ensemble training data: |██████████|100%
Generating ensemble test data: |██████████|100%




Training base predictors on MRI volumes...
        
... for ensemble performance analysis...


Generating ensemble training data: |██████████|100%
Generating ensemble test data: |██████████|100%




Training base predictors on Demo, APOE4 & others...
        
... for ensemble performance analysis...


...

Generating ensemble training data: |██████████|100%
Generating ensemble test data: |██████████|100%




Training base predictors on MRI ROI: Cortical Thickness Average...
        
... for ensemble performance analysis...


Generating ensemble training data: |██████████|100%
Generating ensemble test data: |██████████|100%




Training base predictors on MRI ROI: Cortical Thickness Standard Deviation...
        
... for ensemble performance analysis...


Generating ensemble training data: |██████████|100%
Generating ensemble test data: |██████████|100%


In [14]:
RNN_training_data = [[dfs[0][i] for dfs in meta_data] for i in range(5)]
RNN_test_data = [[dfs[1][i] for dfs in meta_data] for i in range(5)]
RNN_training_data_final = [df[2] for df in meta_data]

In [20]:
RNN_training_data[0][0]

modality       Main cognitive tests                                      \
base predictor                 ADAB       XGB   DT    RF        GB  KNN   
sample                            0         0    0     0         0    0   
0                          0.505486  0.958904  1.0  0.81  0.788702  0.4   
1                          0.500068  0.727477  0.0  0.70  0.417248  0.6   
2                          0.503276  0.916745  1.0  0.78  0.741648  1.0   
3                          0.505286  0.991051  1.0  0.85  0.741844  0.6   
4                          0.496880  0.021519  0.0  0.13  0.216745  0.2   
..                              ...       ...  ...   ...       ...  ...   
556                        0.596718  0.591450  0.0  0.55  0.705602  0.6   
557                        0.509774  0.992870  1.0  0.94  0.865255  1.0   
558                        0.502417  0.898339  1.0  0.67  0.841495  0.8   
559                        0.502839  0.741628  0.0  0.66  0.719824  0.6   
560                        0.496914  0.291351  1.0  0.46  0.404681  0.6   

modality                                                ...  \
base predictor        LR        NB       MLP       SVM  ...   
sample                 0         0         0         0  ...   
0               0.663143  0.492347  0.652059  0.711980  ...   
1               0.740257  0.810885  0.729524  0.717001  ...   
2               0.654685  0.575966  0.636532  0.711819  ...   
3               0.669352  0.498383  0.648012  0.715373  ...   
4               0.659404  0.345839  0.624848  0.722468  ...   
..                   ...       ...       ...       ...  ...   
556             0.686991  0.743053  0.679712  0.716385  ...   
557             0.734773  0.733307  0.767306  0.717208  ...   
558             0.737363  0.801142  0.748774  0.716365  ...   
559             0.693575  0.686652  0.683713  0.714863  ...   
560             0.798245  0.888762  0.724508  0.703333  ...   

modality       MRI ROI: Cortical Thickness Standard Deviation                  \
base predictor                                            XGB    DT        RF   
sample                                                      0     0         0   
0                                                    0.951356  0.00  0.600000   
1                                                    0.888306  0.00  0.620000   
2                                                    0.080768  0.00  0.570000   
3                                                    0.403720  0.00  0.541566   
4                                                    0.737148  1.00  0.495242   
..                                                        ...   ...       ...   
556                                                  0.996372  1.00  0.830000   
557                                                  0.720703  0.72  0.715780   
558                                                  0.996742  1.00  0.895556   
559                                                  0.994618  1.00  0.720000   
560                                                  0.995968  1.00  0.780000   

modality                                                              labels  
base predictor        GB  KNN        LR        NB       MLP       SVM         
sample                 0    0         0         0         0         0         
0               0.787485  1.0  0.662486  0.217019  0.673205  0.711213      0  
1               0.708750  0.8  0.831279  0.501342  0.896478  0.747484      0  
2               0.612974  0.8  0.517669  0.154307  0.455728  0.614581      1  
3               0.147661  1.0  0.621391  0.091220  0.684182  0.708136      0  
4               0.460196  0.8  0.521172  0.074794  0.459600  0.668568      1  
..                   ...  ...       ...       ...       ...       ...    ...  
556             0.884912  0.6  0.763059  0.998963  0.667635  0.734061      0  
557             0.731600  1.0  0.742040  0.282675  0.678986  0.749463      1  
558             0.945541  1.0  0.876282  0.998675  0.864391  0

### make first time point in meta-data multiclass

In [ ]:
def get_column_names(frame):
    column_names = []
    for i in range(frame.columns.nlevels):
        if i == 0:
            column_names.append(frame.columns.get_level_values(i).unique().drop("labels"))
            
        else:
            column_names.append(frame.columns.get_level_values(i).unique().drop(''))
    
    return column_names

In [ ]:
def fix_first_time_point(df):
    new_columns = get_column_names(df)
    classes=[0,1,2]
    new_columns.append(classes)
    new_mux=pd.MultiIndex.from_product(iterables=new_columns, names=["modality", "base predictor", "sample", "class"])
    new_df = pd.DataFrame(columns=new_mux)

    for col in new_df.columns:
        if col[-1] == 0:
            new_df[col] = 1 - df[col[:-1]]
        elif col[-1] == 1:
            new_df[col] = df[col[:-1]]
        else:
            new_df[col] = 0
    
    new_df['labels'] = df['labels']

    return new_df

In [ ]:
for i in range(len(RNN_training_data)):
    RNN_training_data[i][0] = fix_first_time_point(RNN_training_data[i][0])
    RNN_test_data[i][0] = fix_first_time_point(RNN_test_data[i][0])

# TIME SERIES TIME

In [21]:
from keras import backend as K

def ordinal_regression_loss(y_true, y_pred):
    """
    Ordinal regression loss function.

    Parameters:
    - y_true: true labels (ground truth)
    - y_pred: predicted labels

    Returns:
    - loss value
    """
    # Assuming y_true and y_pred are tensors of shape (batch_size, num_classes)

    # Calculate cumulative probabilities for true and predicted labels
    true_cum_probs = K.cumsum(K.softmax(y_true, axis=-1), axis=-1)
    pred_cum_probs = K.cumsum(K.softmax(y_pred, axis=-1), axis=-1)

    # Calculate the ordinal regression loss
    loss = K.sum((true_cum_probs - pred_cum_probs) ** 2)

    return loss

2024-01-18 18:17:31.496225: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-18 18:17:31.522673: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-18 18:17:31.522700: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-18 18:17:31.523491: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-18 18:17:31.528068: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-18 18:17:31.528565: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [ ]:
y_preds = [] # LSTM predictions at every time point. Will be populated by 5 arrays
for i in range(len(RNN_training_data)):
    from keras.models import Sequential
    from keras.layers import LSTM,Dense
    lstm = Sequential()
    lstm.add(LSTM(units=50+10*i, input_shape=(4,240), return_sequences=True))
    lstm.add(Dense(units=3, activation='softmax'))
    lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) #loss='categorical_crossentropy'


    labels_across_time = np.column_stack([df['labels'].values for df in RNN_training_data[i]])
    labels_across_time = np.eye(3)[labels_across_time]

    RNN_training_data_fold = [data.drop(columns=["labels"], axis=1) for data in RNN_training_data[i]]
    data_arrays_per_timepoint = [df.to_numpy() for df in RNN_training_data_fold]
    tensor_3d = np.stack(data_arrays_per_timepoint, axis=0)
    tensor_3d = np.transpose(tensor_3d, (1,0,2))

    lstm.fit(tensor_3d, labels_across_time)

    RNN_test_data_fold = [data.drop(columns=["labels"], axis=1) for data in RNN_test_data[i]]
    data_arrays_per_timepoint_test = [df.to_numpy() for df in RNN_test_data_fold]
    tensor_3d_test = np.stack(data_arrays_per_timepoint_test, axis=0)
    tensor_3d_test = np.transpose(tensor_3d_test, (1,0,2))

    y_preds.append(lstm.predict(tensor_3d_test))

In [ ]:
y_preds_argmax = [np.argmax(pred, axis=-1) for pred in y_preds]

In [ ]:
y_trues = []
for i in range(len(RNN_test_data)):
    y_true = pd.concat([data["labels"] for data in RNN_test_data[i]], axis=1).to_numpy()
    y_trues.append(y_true)


In [ ]:
from sklearn.metrics import classification_report
for i in range(len(y_preds_argmax)):
    print(f" \n FOLD {i+1} \n")
    for j in range(4):
        print(classification_report(y_pred=y_preds_argmax[i][:,j], y_true=y_trues[i][:,j]))

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
dem_f1 = []
for i in range(len(y_preds_argmax)):
    precision, recall, f1, support = precision_recall_fscore_support(y_pred=y_preds_argmax[i][:,-1], y_true=y_trues[i][:,-1])
    dem_f1.append(f1[2])

np.mean(dem_f1), np.std(dem_f1)